In [1]:
from uk_covid19 import Cov19API
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import ipywidgets as wdg

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
def access_api():
    filters = [
        'areaType=overview'
    ]

    structure = {
        "date": "date",
        "first": "cumPeopleVaccinatedFirstDoseByPublishDate",
        "second": "cumPeopleVaccinatedSecondDoseByPublishDate"
    }
    api = Cov19API(filters=filters, structure=structure)
    people=api.get_json()
    with open("people.json", "wt") as file:
        json.dump(people, file)



In [4]:
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")
def wrangle_data():
    global peopledf
    with open("people.json", "rt") as file:
        data=json.load(file)
    datalist = data['data']
    dates=[entry['date'] for entry in datalist ]
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    peopledf=pd.DataFrame(index=index, columns=['first', 'second','total'])
    
    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['first', 'second']:
            if pd.isna(peopledf.loc[date, column]): 
                value= entry[column] if entry[column]!=None else 0.0
                peopledf.loc[date, column]=value
        for column in ['total']:
            value = entry['first'] + entry['second']
            peopledf.loc[date, column]=value
            
    peopledf.fillna(0.0, inplace=True)
    return peopledf



The following form demonstrates the number of people who received the first dose and the second dose by publish date. 

The time range is between 2021-01-10 to 2022-09-11, because PHE only provides those data.

In [35]:
wrangle_data()

,first,second,total
2021-01-10,2286572,391399,2677971
2021-01-11,2431648,412167,2843815
2021-01-12,2639309,428232,3067541
2021-01-13,2918252,437977,3356229
2021-01-14,3234946,443234,3678180
...,...,...,...
2022-09-07,53808854,50750722,104559576
2022-09-08,53809727,50752513,104562240
2022-09-09,53810757,50754555,104565312
2022-09-10,53812756,50760564,104573320


There is a chart to represent those data.

In [14]:
columns=wdg.SelectMultiple(
    options=['first', 'second', 'total'], 
    value=['first', 'second'], 
    rows=3, 
    description='Times',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([columns, scale])


def times_graph(gcolumns,gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcolumns)
    if ncols>0:
        peopledf[list(gcolumns)].plot(logy=logscale) 
        plt.show() 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
output=wdg.interactive_output(times_graph, {'gcolumns': columns,'gscale': scale})

display(controls, output)

Output()

You can click the following button to get the new data from API.

In [31]:
def refresh_graph():
    current=columns.value
    if current==tuple([columns.options[0]]):
        other=tuple([columns.options[1]])
    else:
        other=tuple([columns.options[0]])
    columns.value=other 
    columns.value=current 

In [36]:
def api_button_callback(button):

    apidata=access_api()
    global peopledf
    peopledf=wrangle_data()

    refresh_graph()

    apibutton.icon="check"

    
apibutton=wdg.Button(
    description='refresh',
    disabled=False,
    button_style='info',
    tooltip="Keep calm and carry on",
    icon='refresh'
)

apibutton.on_click(api_button_callback) 

display(apibutton)

Button(button_style='info', description='refresh', icon='refresh', style=ButtonStyle(), tooltip='Keep calm and…